In [ ]:
 import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
train_df = pd.read_csv('UNSW_NB15_training-set.csv')
test_df = pd.read_csv('UNSW_NB15_testing-set.csv')

In [ ]:
categorical_features = ['proto', 'service', 'state']
numeric_features = train_df.columns.difference(categorical_features + ['label', 'attack_cat'])

In [ ]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # 忽略未知类别
])

X_train = preprocessor.fit_transform(train_df).toarray()
y_train = train_df['label'].values.reshape(-1, 1)
X_test = preprocessor.transform(test_df).toarray()
y_test = test_df['label'].values.reshape(-1, 1)

In [ ]:
from pyESN import ESN

n_inputs = X_train.shape[1]
n_outputs = 1

esn = ESN(
    n_inputs=n_inputs,
    n_outputs=n_outputs,
    n_reservoir=500,
    spectral_radius=0.95,
    sparsity=0.2,
    noise=0.001,
    teacher_scaling=1.0,
    teacher_forcing=True,
    random_state=42,
    silent=False
)

In [ ]:
pred_train = esn.fit(X_train, y_train, inspect=False)

harvesting states...
fitting...
training error:
0.15325368814850723


In [ ]:
pred_test = esn.predict(X_test, continuation=True)
y_pred = (pred_test > 0.5).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print(f"accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

accuracy: 0.8655
              precision    recall  f1-score   support

           0       0.94      0.75      0.83     37000
           1       0.82      0.96      0.89     45332

    accuracy                           0.87     82332
   macro avg       0.88      0.85      0.86     82332
weighted avg       0.88      0.87      0.86     82332

